# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [14]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head(10)


,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date,Country
0,0,busselton,-33.65,115.33,70.00,64,24,8.05,1607621883,AU
1,1,tuktoyaktuk,69.45,-133.04,-11.20,83,90,12.75,1607621883,CA
2,2,ushuaia,-54.80,-68.30,62.60,39,20,10.29,1607621883,AR
3,3,amapa,1.00,-52.00,92.57,42,28,7.02,1607621883,BR
4,4,rikitea,-23.12,-134.97,76.66,80,49,13.87,1607621883,PF
5,5,prokhorovka,51.04,36.73,21.20,67,0,13.42,1607621883,RU
6,6,castro,-24.79,-50.01,74.03,66,61,3.29,1607621884,BR
7,7,albany,42.60,-73.97,42.01,55,49,4.00,1607621849,US
8,8,east london,-33.02,27.91,69.80,83,20,9.17,1607621884,ZA
9,9,fairbanks,64.84,-147.72,1.99,84,90,1.12,1607621672,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps first

gmaps.configure(api_key = g_key)

#enable widgets to visualize map below
#!jupyter nbextension enable --py gmaps
#!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [26]:
locations = city_data_df[["Latitude","Longitude"]]
humidity = city_data_df["Humidity"]

#create base map and define zoom_level/center so an appropriate size map appears...otherwise I get 3 copies of the world map side-by-side
fig = gmaps.figure(zoom_level = 2, center = (0,0))

#add heat layer
humidity_heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, max_intensity=100, dissipating=False, point_radius = 4)

fig.add_layer(humidity_heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [64]:
#My favorability criteria are: 
#(1) humidity less than or equal to 60%;
#(2) cloudiness between 20% and 60%, inclusive and;
#(3) wind speeds less than 10 mph

favorable_cities_df = city_data_df.loc[(city_data_df["Humidity"] <= 60) & \
                                       (city_data_df["Cloudiness"] >= 20) & (city_data_df["Cloudiness"] <= 60) & \
                                       (city_data_df["Wind Speed"] < 10) ].dropna()


#favorable_cities_df = city_data_df.loc[(city_data_df["Humidity"] <= 60)]
#favorable_cities_df = favorable_cities_df.loc[(city_data_df["Cloudiness"] >= 20) & (city_data_df["Cloudiness"] <= 60)]
#favorable_cities_df = favorable_cities_df.loc[(city_data_df["Wind Speed"]) < 10].dropna()




favorable_cities_df.head(12)

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date,Country
3,3,amapa,1.00,-52.00,92.57,42,28,7.02,1607621883,BR
7,7,albany,42.60,-73.97,42.01,55,49,4.00,1607621849,US
117,117,progreso,21.28,-89.67,77.00,50,20,9.17,1607621766,MX
136,136,carauari,-4.88,-66.90,91.51,51,60,4.23,1607621784,BR
201,201,mirabad,30.44,61.84,51.96,43,44,5.35,1607621914,AF
206,206,taoudenni,22.68,-3.98,66.88,22,21,9.51,1607621915,ML
262,262,ghanzi,-21.57,21.78,86.99,31,60,6.20,1607621924,BW
280,280,sipe sipe,-17.45,-66.38,77.00,25,40,3.36,1607621927,BO
307,307,marsh harbour,26.54,-77.06,68.77,54,55,6.29,1607621720,BS
340,340,yacuiba,-22.04,-63.68,98.60,39,20,9.17,1607621940,BO


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
